In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.signal import find_peaks

In [ ]:
#input/output directories
base_path = r"C:\Users\lenovo\thesis\classification\csv"
output_path=  r"C:\Users\lenovo\thesis\classification\wavelets"

input_files = [
    output_path+"\\B007_1_123\\X123_DE_time.csv",      
    output_path+"\\B014_1_190\\B014_1_190X190_DE_time.csv",
    output_path+"\\B021_1_227\\B021_1_227X227_DE_time.csv",
    
    output_path+"\\OR007_6_1_136\\X136_DE_time.csv",
    output_path+"\\OR014_6_1_202\\OR014_6_1_202X202_DE_time.csv",
    output_path+"\\OR021_6_1_239\\OR021_6_1_239X239_DE_time.csv",

    output_path+"\\IR007_1_110\\X110_DE_time.csv",
    output_path+"\\IR014_1_175\\IR014_1_175X175_DE_time.csv",
    output_path+"\\IR021_1_214\\IR021_1_214X214_DE_time.csv",
    
]

output_directories = [
   output_path+"\\bopolar7",
   output_path+"\\bopolar14"
   output_path+"\\bopolar21"
   output_path+"\\orpolar7",
   output_path+"\\orpolar14"
   output_path+"\\orpolar21"
   output_path+"\\irpolar7",
   output_path+"\\irpolar14"
   output_path+"\\irpolar21"
]


In [ ]:
# calculate frequency and amplitute to plot the wavelets using FFT
#https://github.com/yellowsimulator/bearing_watcher/tree/main/bearing_watcher

def calculate_spectrum(data, fs):
    fft_result = np.fft.fft(data)
    
    freqs = np.fft.fftfreq(len(data), 1 / fs)    
    amplitude = np.abs(fft_result)

    #we use only half
    freqs = freqs[:len(freqs)//2]
    amplitude = amplitude[:len(amplitude)//2]
    
    return freqs, amplitude

In [ ]:
#plot frequncy spectrum based on peaks
def plot_frequency_spectrum(freqs, amplitude, output_path):
    peaks, _ = find_peaks(amplitude, height=0)
    
    fig, ax = plt.subplots()
    ax.plot(freqs, amplitude)
    ax.set_title("Frequency Spectrum")
    ax.set_xlabel("Frequency (Hz)")
    ax.set_ylabel("Amplitude")
    
    plt.savefig(output_path, dpi=300)
    plt.close(fig)

In [ ]:
#plot based on chunks
def process_chunk(chunk_data, output_path):
    fs = 48000
    freqs, amplitude = calculate_spectrum(chunk_data, fs)
    plot_frequency_spectrum(freqs, amplitude, output_path)

In [ ]:
#split the signals to samples with 4800 datapoints with 50% overlap

def process_dataset(input_file,  output_directory, chunk_size=4800, overlap=0.5):
    fs = 48000

    dataset = pd.read_csv(input_file, header=None)
    data = dataset.iloc[:, 0].values 

    nsamples = len(data)
    overlaps = int(chunk_size * overlap)
    nchunks = (nsamples - chunk_size) // overlaps + 1

    os.makedirs(output_directory, exist_ok=True)

    for j in range(nchunks):
        startid = j * overlaps
        endid = startid + chunk_size
        chunk_data = data[startid:endid]

        file_name = f"chunk{j + 1}_frequency_spectrum.png"
        output_path = os.path.join(output_directory, file_name)

        # create polars
        process_chunk(chunk_data, output_path)

In [ ]:
for input_file, output_directory in zip(input_files, output_directories):
    process_dataset(input_file, output_directory, chunk_size=4800, overlap=0.5)
